In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import json
import polars as pl
from polars import col, lit
import altair as alt
import simrs
from history import History, tabular
import input_builder as b

In [4]:
# Agents
agent_ids = ["a" +str(i) for i in range(1, 5)]

# Goods
wheat = "Wheat"
goods = [wheat]

# Ports
genoa = "Genoa"
rome = "Rome"
milan = "Milan"
venice = "Venice"
marsailles = "Marsailles"
port_ids = [genoa, rome, milan, venice, marsailles]

# genoa <-> milan <-> marsailles
#   |         |
# rome <-> venice
edges = [(genoa, milan), (milan, marsailles), (rome, venice), (rome, genoa), (venice, milan)]
_market = lambda net: b._market(wheat, b._market_info(net=net)) 

# net balanced
ports = [
  b._port(genoa, _market(100)), 
  b._port(milan, _market(50)), 
  b._port(marsailles, _market(-100)), 
  b._port(rome, _market(0)), 
  b._port(venice, _market(-50))
]

_agent = lambda id, pos: b._agent(id, pos, 100, "Random")
agents = [_agent(id, port_id) for (id, port_id) in zip(agent_ids, port_ids)]

x = b._inputFormat( agents = agents, ports=ports, edges= edges, opts=b._opts(ticks=10))
history = simrs.run(x)
(actions, agents, markets) = tabular(history)

In [ ]:
markets.select(col("pricer").struct.field("base_price")).head()
markets.select(col("pricer")).schema

In [ ]:
markets.schema["pricer"]

Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)